# 2F2L on MNIST

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nanboy-Ronan/2F2L-Federated-Unlearning/blob/main/notebooks/mnist.ipynb)

This notebook demos how 2F2L works on MNIST. For direct observation of the effect of unlearning, we adapt backdoor attacks as describted in our paper.

In [1]:
!nvidia-smi

Fri Sep 20 16:44:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              16W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Clone the Repo

In [2]:
!git clone https://github.com/Nanboy-Ronan/2F2L-Federated-Unlearning.git
%cd 2F2L-Federated-Unlearning

Cloning into '2F2L-Federated-Unlearning'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 51 (delta 12), reused 46 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 243.02 KiB | 9.72 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/2F2L-Federated-Unlearning


## Download data and pre-trained checkpoints

This dataset and pretrained checkpoint contains all pre-processed datasets and pre-trained models in our paper.

In [3]:
!wget -nc https://object-arbutus.cloud.computecanada.ca:443/rjin/2F2L/data.zip

--2024-09-20 16:45:01--  https://object-arbutus.cloud.computecanada.ca/rjin/2F2L/data.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16492177175 (15G) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  15.36G   103MB/s    in 2m 22s  

2024-09-20 16:47:23 (111 MB/s) - ‘data.zip’ saved [16492177175/16492177175]



In [4]:
!wget -nc https://object-arbutus.cloud.computecanada.ca:443/rjin/2F2L/checkpoint.zip

--2024-09-20 16:47:24--  https://object-arbutus.cloud.computecanada.ca/rjin/2F2L/checkpoint.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3811648030 (3.5G) [application/zip]
Saving to: ‘checkpoint.zip’

checkpoint.zip      100%[===================>]   3.55G   111MB/s    in 33s     

2024-09-20 16:47:56 (112 MB/s) - ‘checkpoint.zip’ saved [3811648030/3811648030]



## Setup the environment

In [5]:
!pip install git+https://github.com/openai/CLIP.git
!pip install salesforce-lavis

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-nw43z1jd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-nw43z1jd
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [6]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.11.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
braceexpand==0.1.7
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.7.4
cffi==1.16.0
cfgv==3.4.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
clip @ git+https://github.com/openai/CLIP.git@dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==

## Unzip Data and Checkpoint

In [7]:
!unzip data.zip
%rm -f data.zip

Archive:  data.zip
   creating: data/
  inflating: data/Domainnet.pth      
   creating: data/FashionMNIST/
   creating: data/FashionMNIST/raw/
  inflating: data/FashionMNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz  
  inflating: data/FashionMNIST/raw/t10k-labels-idx1-ubyte  
 extracting: data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz  
  inflating: data/FashionMNIST/raw/train-images-idx3-ubyte  
  inflating: data/FashionMNIST/raw/train-images-idx3-ubyte.gz  
  inflating: data/FashionMNIST/raw/train-labels-idx1-ubyte  
 extracting: data/FashionMNIST/raw/train-labels-idx1-ubyte.gz  
   creating: data/MNIST/
   creating: data/MNIST/raw/
  inflating: data/MNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/MNIST/raw/t10k-images-idx3-ubyte.gz  
  inflating: data/MNIST/raw/t10k-labels-idx1-ubyte  
 extracting: data/MNIST/raw/t10k-labels-idx1-ubyte.gz  
  inflating: data/MNIST/raw/train-images-idx3-ubyte  
  inflating: data/MNIST/raw/train-

In [8]:
!unzip checkpoint.zip
%rm -f checkpoint.zip

Archive:  checkpoint.zip
   creating: checkpoint/warmup_ce/
   creating: checkpoint/warmup_ce/mnist/
  inflating: checkpoint/warmup_ce/mnist/bs_32_lr_0.001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/cifar10/
  inflating: checkpoint/warmup_ce/cifar10/bs_32_lr_0.001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/20clsimgnet_clipvit/
  inflating: checkpoint/warmup_ce/20clsimgnet_clipvit/bs_32_lr_0.0001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/domainnet_clipvit/
  inflating: checkpoint/warmup_ce/domainnet_clipvit/bs_32_lr_0.0001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/flowers_blip2/
  inflating: checkpoint/warmup_ce/flowers_blip2/bs_32_lr_0.001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  


## Pretrain

A pre-trained model can help make better taylor expansion for linear training. We offer some pre-trained models in the checkpoint and pretrained_models folder. If you want to do the pre-train yourself, here is two examples of how to launch them.

In [ ]:
# The pretraining for MNIST and CIFAR is provided here. Uncomment them to execute.
# !python pretrain.py --warmup_batch_size 32 --warmup_lr 0.001 --data_dir ./data --save_freq 10 --max_server_rate 0.1 --server_rate 0.1 --dataset mnist --num_users 5
# For CIFAR and other models, it's best to start from a model pretrained from imgnet like the one below.
# !python pretrain.py --warmup_batch_size 32 --warmup_lr 0.001 --data_dir ./data --save_freq 5 --server_rate 0.1 --max_server_rate 0.1 --num_users 5 --log --dataset cifar10 --imgnet_path ./pretrained_models/NTKCIFAR_best.pt

## Federated Learning (FLT)

In [ ]:
!python flt.py --warmup_lr 0.001 --lr 0.0001 --data_dir ./data --server_rate 0.1 --eval_from 50 --train_epochs 400 --dataset mnist --mode mixlinear --remove_idx 0 --num_users 5

2024-09-17 02:55:13.281800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 02:55:13.281854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 02:55:13.283550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 02:55:13.292145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 02:55:14.290525: W tensorflow/compiler/tf2

The test acc is around 96% and the backdoor success rate is around 94%.

## Federated Unlearning (FedRemoval)

In [ ]:
!python fedremoval.py --warmup_lr 0.001 --lr 0.0001 --weight_decay 0.01 --data_dir ./data --server_rate 0.1 --max_server_rate 0.1 --eval_from 50 --num_epochs 3200 --train_epochs 400 --removal_lr 0.01 --dataset mnist --mode mixlinear --remove_idx 0 --num_users 5

Streaming output truncated to the last 5000 lines.
2024-09-17 07:03:27,569 - INFO - Removal Loss:         2.699404
2024-09-17 07:03:29,268 - INFO - Test Loss: 0.0707 | Test Acc: 89.7436%
2024-09-17 07:03:32,949 - INFO -  0   | Server Backdoor Test Loss: 1.6859 | Server Backdoor Success Rate: 5.7864%
2024-09-17 07:03:32,949 - INFO - ============ Removal train epoch 1291 ============
2024-09-17 07:03:34,199 - INFO - Removal Loss:         2.695599
2024-09-17 07:03:34,199 - INFO - ============ Removal train epoch 1292 ============
2024-09-17 07:03:35,484 - INFO - Removal Loss:         2.689158
2024-09-17 07:03:35,484 - INFO - ============ Removal train epoch 1293 ============
2024-09-17 07:03:36,786 - INFO - Removal Loss:         2.684514
2024-09-17 07:03:36,786 - INFO - ============ Removal train epoch 1294 ============
2024-09-17 07:03:38,054 - INFO - Removal Loss:         2.676161
2024-09-17 07:03:38,054 - INFO - ============ Removal train epoch 1295 ============
2024-09-17 07:03:39,330

After unlearning, the test accuracy is 95%, very close to the one before unlearning (96%). THe backdoor sccuess rate decreases to 10%, compared to previously 94%.